# Análisis de Flujos de Dinero en Ekhilur

Este notebook realiza un análisis detallado de las transacciones financieras en la plataforma Ekhilur, utilizando consultas directas a la base de datos MySQL.

## 1. Configuración Inicial

Primero importamos las librerías necesarias y establecemos la conexión con la base de datos.

In [1]:
# Importar librerías necesarias
import mysql.connector
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Configuración de visualización
plt.style.use('seaborn')
sns.set_palette('husl')
%matplotlib inline

# Configuración para mostrar todas las columnas
pd.set_option('display.max_columns', None)

C:\Users\mikel\AppData\Local\Temp\ipykernel_29496\1802020149.py:12: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


### Función para conectar a la base de datos

Creamos una función helper para manejar las consultas SQL.

In [2]:
import os
from dotenv import load_dotenv
import mysql.connector
import pandas as pd

# Cargar variables de entorno
load_dotenv()

def get_db_connection():
    return mysql.connector.connect(
        host='localhost',
        port=3308,
        user='user',
        password='userpassword',
        database='ekhilur'
    )

def ejecutar_query(query):
    """Ejecuta una query SQL y devuelve los resultados como DataFrame"""
    conn = get_db_connection()
    try:
        return pd.read_sql_query(query, conn)
    finally:
        conn.close()

## 2. Análisis de Tipos de Operaciones

Analizamos la distribución de las operaciones y sus importes asociados.

In [3]:
# Consulta para obtener los tipos de operaciones y sus estadísticas
query_operaciones = """
SELECT 
    o.Operacion,
    COUNT(*) as total_operaciones,
    SUM(f.Cantidad) as suma_total,
    AVG(f.Cantidad) as promedio_importe
FROM fact_table f
JOIN dim_operaciones o ON f.Id_tipo_operacion = o.Id_tipo_operacion
GROUP BY o.Operacion
ORDER BY suma_total DESC;
"""

df_operaciones = ejecutar_query(query_operaciones)

# Crear gráfico de barras con Plotly
fig = go.Figure(data=[
    go.Bar(name='Suma Total', x=df_operaciones['Operacion'], y=df_operaciones['suma_total'])
])

fig.update_layout(
    title='Volumen Total por Tipo de Operación',
    xaxis_title='Tipo de Operación',
    yaxis_title='Suma Total (€)',
    template='plotly_white'
)

fig.show()

C:\Users\mikel\AppData\Local\Temp\ipykernel_29496\1450764005.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)


## 3. Análisis Temporal de Transacciones

Analizamos cómo evolucionan las transacciones a lo largo del tiempo.

In [4]:
# Consulta para análisis temporal
query_temporal = """
SELECT 
    DATE(CONCAT(
        SUBSTRING(f.Id_fecha, 1, 4), '-',
        SUBSTRING(f.Id_fecha, 5, 2), '-',
        SUBSTRING(f.Id_fecha, 7, 2)
    )) as fecha,
    o.Operacion,
    SUM(f.Cantidad) as suma_diaria
FROM fact_table f
JOIN dim_operaciones o ON f.Id_tipo_operacion = o.Id_tipo_operacion
GROUP BY fecha, o.Operacion
ORDER BY fecha;
"""

df_temporal = ejecutar_query(query_temporal)

# Crear gráfico de líneas temporal
fig = px.line(df_temporal, 
              x='fecha', 
              y='suma_diaria', 
              color='Operacion',
              title='Evolución Temporal de Transacciones por Tipo')

fig.update_layout(
    xaxis_title='Fecha',
    yaxis_title='Importe Total Diario (€)',
    template='plotly_white'
)

fig.show()

C:\Users\mikel\AppData\Local\Temp\ipykernel_29496\1450764005.py:22: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [5]:
# Consulta para verificar consistencia de datos
query_consistencia = """
WITH stats AS (
    SELECT 
        COUNT(*) as total_registros,
        COUNT(DISTINCT Id_fecha) as dias_unicos,
        COUNT(DISTINCT Id_tipo_operacion) as tipos_operacion_unicos,
        COUNT(DISTINCT Usuario_emisor) as emisores_unicos,
        COUNT(DISTINCT Usuario_receptor) as receptores_unicos,
        SUM(Cantidad) as suma_total,
        AVG(Cantidad) as promedio_operacion,
        MIN(Cantidad) as min_cantidad,
        MAX(Cantidad) as max_cantidad,
        COUNT(CASE WHEN Cantidad < 0 THEN 1 END) as operaciones_negativas,
        COUNT(CASE WHEN Cantidad > 0 THEN 1 END) as operaciones_positivas,
        COUNT(CASE WHEN Cantidad = 0 THEN 1 END) as operaciones_cero
    FROM fact_table
),
balance_por_tipo AS (
    SELECT 
        o.Operacion,
        COUNT(*) as num_operaciones,
        SUM(f.Cantidad) as total_por_tipo,
        AVG(f.Cantidad) as promedio_por_tipo,
        MIN(f.Cantidad) as min_por_tipo,
        MAX(f.Cantidad) as max_por_tipo
    FROM fact_table f
    JOIN dim_operaciones o ON f.Id_tipo_operacion = o.Id_tipo_operacion
    GROUP BY o.Operacion
),
balance_por_fecha AS (
    SELECT 
        SUBSTRING(Id_fecha, 1, 4) as año,
        COUNT(*) as operaciones_por_año,
        SUM(Cantidad) as total_por_año
    FROM fact_table
    GROUP BY SUBSTRING(Id_fecha, 1, 4)
)
SELECT 
    'Estadísticas Generales' as categoria,
    total_registros,
    dias_unicos,
    tipos_operacion_unicos,
    emisores_unicos,
    receptores_unicos,
    suma_total,
    promedio_operacion,
    min_cantidad,
    max_cantidad,
    operaciones_negativas,
    operaciones_positivas,
    operaciones_cero
FROM stats;
"""

# Ejecutar la consulta
df_consistencia = ejecutar_query(query_consistencia)

# Mostrar resultados
print("\n=== ANÁLISIS DE CONSISTENCIA DE DATOS ===")
print("\nEstadísticas Generales:")
print("-" * 50)
for columna in df_consistencia.columns:
    if columna != 'categoria':
        valor = df_consistencia[columna].iloc[0]
        if isinstance(valor, (int, float)):
            if columna in ['suma_total', 'promedio_operacion', 'min_cantidad', 'max_cantidad']:
                print(f"{columna}: {valor:,.2f} €")
            else:
                print(f"{columna}: {valor:,}")
        else:
            print(f"{columna}: {valor}")

# Consulta adicional para balance por tipo de operación
query_balance_tipo = """
SELECT 
    o.Operacion,
    COUNT(*) as num_operaciones,
    SUM(f.Cantidad) as total_por_tipo,
    AVG(f.Cantidad) as promedio_por_tipo,
    MIN(f.Cantidad) as min_por_tipo,
    MAX(f.Cantidad) as max_por_tipo
FROM fact_table f
JOIN dim_operaciones o ON f.Id_tipo_operacion = o.Id_tipo_operacion
GROUP BY o.Operacion
ORDER BY total_por_tipo DESC;
"""

df_balance_tipo = ejecutar_query(query_balance_tipo)

print("\nBalance por Tipo de Operación:")
print("-" * 50)
print(df_balance_tipo.to_string(index=False))

# Consulta para balance por año
query_balance_anual = """
SELECT 
    SUBSTRING(Id_fecha, 1, 4) as año,
    COUNT(*) as operaciones_por_año,
    SUM(Cantidad) as total_por_año
FROM fact_table
GROUP BY SUBSTRING(Id_fecha, 1, 4)
ORDER BY año;
"""

df_balance_anual = ejecutar_query(query_balance_anual)

print("\nBalance por Año:")
print("-" * 50)
print(df_balance_anual.to_string(index=False))

# Visualización del balance por tipo de operación
fig = go.Figure(data=[
    go.Bar(
        name='Total por Tipo',
        x=df_balance_tipo['Operacion'],
        y=df_balance_tipo['total_por_tipo'],
        text=df_balance_tipo['total_por_tipo'].round(2),
        textposition='auto',
    )
])

fig.update_layout(
    title='Balance Total por Tipo de Operación',
    xaxis_title='Tipo de Operación',
    yaxis_title='Total (€)',
    template='plotly_white'
)

fig.show()

# Visualización del balance anual
fig2 = go.Figure(data=[
    go.Bar(
        name='Total por Año',
        x=df_balance_anual['año'],
        y=df_balance_anual['total_por_año'],
        text=df_balance_anual['total_por_año'].round(2),
        textposition='auto',
    )
])

fig2.update_layout(
    title='Balance Total por Año',
    xaxis_title='Año',
    yaxis_title='Total (€)',
    template='plotly_white'
)

fig2.show()

C:\Users\mikel\AppData\Local\Temp\ipykernel_29496\1450764005.py:22: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.




=== ANÁLISIS DE CONSISTENCIA DE DATOS ===

Estadísticas Generales:
--------------------------------------------------
total_registros: 340322
dias_unicos: 366
tipos_operacion_unicos: 12
emisores_unicos: 1530
receptores_unicos: 1339
suma_total: 6,500,191.99 €
promedio_operacion: 19.10 €
min_cantidad: 0.00 €
max_cantidad: 15,000.00 €
operaciones_negativas: 0
operaciones_positivas: 340244
operaciones_cero: 78

Balance por Tipo de Operación:
--------------------------------------------------
              Operacion  num_operaciones  total_por_tipo  promedio_por_tipo  min_por_tipo  max_por_tipo
         Pago a usuario           112779      3430730.90          30.419944          0.00       2060.00
         Conversión a €              969      1267647.50        1308.201754          1.00      15000.00
    Recarga por tarjeta             9306      1255837.00         134.949173         50.00        500.00
         Cobro desde QR            12894       299646.69          23.239235          0.01 

C:\Users\mikel\AppData\Local\Temp\ipykernel_29496\1450764005.py:22: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.




Balance por Año:
--------------------------------------------------
 año  operaciones_por_año  total_por_año
2024               340322     6500191.99
